In [110]:
import math
import sys
from pathlib import Path
import itertools
import pickle
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'
px.defaults.template = 'ggplot2'
cdr_colors = ["#74D2F5", "#5495D6", "#3763F5", "#CF81EB", "#E564F5", "#A04AD6"]
fig.update_layout(
    font_size=16,
    width=500,
    height=500
)
# fig.write_image('image.pdf')

from collections import namedtuple
ResiCount = namedtuple('ResiCount', ['antibody', 'antigen'])
TYRs = namedtuple('TYRs', ['heavy', 'light', 'antigen'])
AA_count = namedtuple('AA_count', ['heavy', 'light', 'antigen'])
PiPiPair = namedtuple('PiPiPair', ['antibody', 'antigen'])
PionPair = namedtuple('PionPair', ['ring', 'ion'])
HBondAtom = namedtuple('HBondAtom', ['chainID', 'chain_type',
                       'CDR', 'resSeq', 'resname', 'index', 'serial', 'element', 'is_sidechain'])
HBond = namedtuple('HBond', ['donor', 'acceptor'])
ShieldingAtom = namedtuple(
    'ShieldingAtom', ['chainID', 'chain_type', 'CDR', 'resSeq', 'resname', 'index',
                      'serial', 'element', 'is_sidechain'])
PolarCount = namedtuple('PolarCount', ['cdr_SC', 'cdr_BB', 'epi_SC', 'epi_BB'])
Chains = namedtuple('Chains', ['antibody', 'antigen'])
AA_LIST = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS",
           "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]
from collections import Counter

source_location = Path().resolve()
sys.path.append(source_location)

from scripts.abag_interactions_hydrophobic import *
from scripts.abag_interactions_rings import *
from scripts.more_utils import *

casa_dir = Path("/home/pbarletta/labo/22/AbAgInterface")
str_dir = Path.joinpath(casa_dir, "structures/raw")

In [2]:
with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
    filenames = pickle.load(file)

pdb_list = list(filenames.keys())
df_dataset = get_df_dataset(casa_dir)

SabDab protein antigen:
1154 proteins out of 2017, 57.2%
All: 1154
No Hchain: 0
No Lchain: 0
Both chains: 1154
Buried surfaces of 2492 proteins
with both chains: 867


------

## Hydrophobic

In [3]:
with open(Path.joinpath(casa_dir, 'data', 'hydrophobic.pkl'), 'rb') as file:
    df_hydrophobic_atom_indices, df_hydrophobic_atom_serials,\
    df_hydrophobic_resSeq, df_hydrophobic_resname, df_hydrophobic_chain_ID,\
    df_hydrophobic_chain_type, df_hydrophobic_cdr = pickle.load(file)

In [177]:
chain_type_dict = {'H': 0, 'K': 0, 'L': 0}
cdr_dict = {'H1': 0, 'H2': 0, 'H3': 0, 'L1': 0, 'L2': 0, 'L3': 0, 
'K1': 0, 'K2': 0, 'K3': 0, 'H0': 0, 'K0': 0, 'L0': 0}
cdr_dict_big_cluster = {'H1': 0, 'H2': 0, 'H3': 0, 'L1': 0, 'L2': 0, 'L3': 0, 
'K1': 0, 'K2': 0, 'K3': 0, 'H0': 0, 'K0': 0, 'L0': 0}
cdr_dict_big_cluster_aa_comp_atm = dict(zip(AA_LIST, itertools.repeat(0, 20)))
cdr_dict_big_cluster_aa_comp_res = dict(zip(AA_LIST, itertools.repeat(0, 20)))
size_largest_cluster_res = []
size_largest_cluster_atm = []
clusters_count = []
cdr_count_largest_cluster = []

for pdb_idcode in pdb_list:
    chain_type_of_each_contact = (df_hydrophobic_chain_type.query(
        f"idcode == '{pdb_idcode}'").chain_type)[0]
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    cdr_of_each_contact = (df_hydrophobic_cdr.query(
        f"idcode == '{pdb_idcode}'").CDR)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname[0])
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq[0])
    if isinstance(chain_type_of_each_contact, float):
        # Testing if is NaN
        print(f"-- BAD: {pdb_idcode} -- ")
        continue
    
    
    nbr_of_clusters = 0
    flag = True
    list_chain_type_cdr = []
    for cluster_cdr, cluster_chainID, cluster_chain_type,\
        cluster_resname, cluster_resSeq in\
        zip(cdr_of_each_contact, chainID_of_each_contact,
        chain_type_of_each_contact, resname_of_each_contact, resSeq_of_each_contact):
        # Only count clusters involving more than 4 carbons
        nbr_of_clusters += 1 if len(cluster_cdr) > 4 else 0
        for contact_cdr, contact_chain_type, contact_resname, contact_resSeq in\
            zip(cluster_cdr, cluster_chain_type, cluster_resname,
            cluster_resSeq):
            if contact_chain_type != '':
                # antibody carbons
                chain_type_dict[contact_chain_type] += 1
                
                chain_type_cdr = contact_chain_type + str(contact_cdr)
                cdr_dict[chain_type_cdr] += 1
                
                list_chain_type_cdr.append(chain_type_cdr)
        if flag:
            if nbr_of_clusters == 0:
                continue
            # Clusters are sorted by size, so the 1st one is the largest.
            for cdr, count in Counter(list_chain_type_cdr).items():
                cdr_dict_big_cluster[cdr] += count
            # Count aminoacid types, at an atomic level:
            for resname in cluster_resname:
                cdr_dict_big_cluster_aa_comp_atm[resname] += 1
            # Count aminoacid types, at residue level:
            # Now I have to make sure I'm counting residues, instead of atoms.
            resi_list = []
            # using both resSeq and chainID to identify each residue,
            # since there're chainID overlaps
            for chain, res in zip (cluster_chainID, cluster_resSeq):
                resi_list.append(chain+str(res))

            unique_resi_list = set(resi_list)
            for unique_ in unique_resi_list:
                # Discarding the chainID for casting
                unique_resSeq = int(unique_[1:])
                # Find the positions where this residue shows up 
                # and only keep the 1st one.
                unique_index = np.where(cluster_resSeq == unique_resSeq)[0][0]
                # Finally, count it
                cdr_dict_big_cluster_aa_comp_res[cluster_resname[unique_index]] += 1

            size_largest_cluster_res.append(len(unique_resi_list))
            size_largest_cluster_atm.append(len(cluster_cdr))
            # Count the number of CDRs in the largest cluster
            cdrs_in_big_cluster = []
            for par in zip(cluster_chain_type, [ str(i) for i in cluster_cdr]):
                putative_cdr = ''.join(par)
                if putative_cdr in cdr_dict.keys():
                    cdrs_in_big_cluster.append(putative_cdr)
            cdr_count_largest_cluster.append(len(set(cdrs_in_big_cluster)))
            flag = False
    if nbr_of_clusters > 0:
        clusters_count.append(nbr_of_clusters)

### origin_of_carbons_in_hydrophobic_interactions

In [180]:
fig = go.Figure(data = go.Pie(labels = ['H1', 'H2', 'H3', 'L1', 'L2', 'L3'], 
    values = [cdr_dict['H1'], cdr_dict['H2'], cdr_dict['H3'],
    cdr_dict['L1']+cdr_dict['K1'], cdr_dict['L2']+cdr_dict['K2'],
    cdr_dict['L3']+cdr_dict['L3']], hole = .4) )
fig.update_traces(marker={'colors': cdr_colors})

### origin_of_the_carbons_from_the_biggest_hydrophobic_cluster

In [179]:
fig = go.Figure(data = go.Pie(labels = ['H1', 'H2', 'H3', 'L1', 'L2', 'L3'], 
    values = [cdr_dict_big_cluster['H1'], cdr_dict_big_cluster['H2'],
    cdr_dict_big_cluster['H3'], cdr_dict_big_cluster['L1']+cdr_dict_big_cluster['K1'],
    cdr_dict_big_cluster['L2']+cdr_dict_big_cluster['L2'],
    cdr_dict_big_cluster['L3']+cdr_dict_big_cluster['K3']], hole = .4))
fig.update_traces(marker={'colors': cdr_colors})

### number_of_clusters_per_PDB

In [7]:
figu = px.histogram(pd.DataFrame({'count':clusters_count}),
    histnorm = 'probability', x='count', labels={'count': "# of clusters"})
figu.update_xaxes(tick0 = 1, dtick=1)

### size_of_the_largest_cluster_of_each_PDB_by_atm

### size_of_the_largest_cluster_of_each_PDB_by_res

In [9]:
xbins = range(0, 45, 4)
counts, bins = np.histogram(size_largest_cluster_res, bins = xbins)
mid_bins = .5 * (bins[:-1] + bins[1:])

figu = px.bar(x=mid_bins, y=counts,
labels={'x': "# of residues on largest cluster", 'y': "Count"})
figu.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = list(mid_bins),
        ticktext = list(mid_bins)
    )
)
figu.update_layout(bargap = .01)
figu.show()

### resname_of_the_carbons_from_the_biggest_hydrophobic_cluster_by_atom

In [10]:
df_aa_comp = pd.DataFrame({'AA': cdr_dict_big_cluster_aa_comp_atm.keys(),
    'Count': cdr_dict_big_cluster_aa_comp_atm.values()})
    
fig_ring = px.histogram(df_aa_comp, x='AA', y='Count', 
    # color='color', barmode='group',
    height=400)
fig_ring.update_xaxes(title = "AA of the atom")
fig_ring.update_yaxes(title = "Count")

In [11]:
with open(Path.joinpath(casa_dir, 'data', 'count_resi.pkl'), 'rb') as file:
    count_resi = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_tot.pkl'), 'rb') as file:
    count_tot = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_tot_pdb.pkl'), 'rb') as file:
    count_tot_pdb = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_resi_pdb.pkl'), 'rb') as file:
    count_resi_pdb = pickle.load(file)

### resname_of_the_carbons_from_the_biggest_hydrophobic_cluster_by_res

In [12]:
df_aa_comp = pd.DataFrame({'AA': cdr_dict_big_cluster_aa_comp_res.keys(),
    'Count': cdr_dict_big_cluster_aa_comp_res.values()})
    
fig_ring = px.histogram(df_aa_comp, x='AA', y='Count', 
    # color='color', barmode='group',
    height=400)
fig_ring.update_xaxes(title = "AA of the atom")
fig_ring.update_yaxes(title = "Count")

### resname_of_the_carbons_from_the_biggest_hydrophobic_cluster_by_res_norm

In [13]:
b = np.array(list(cdr_dict_big_cluster_aa_comp_res.values()))

df_aa_comp = pd.DataFrame({'AA': cdr_dict_big_cluster_aa_comp_res.keys(),
    'Count': b / sum(b)})
    
fig_ring = px.histogram(df_aa_comp, x='AA', y='Count', 
    # color='color', barmode='group',
    height=400)
fig_ring.update_xaxes(title = "AA of the atom")
fig_ring.update_yaxes(title = "Probability", range = [0, .15])

### resname_of_the_carbons_from_the_biggest_hydrophobic_cluster_by_res_ratio_to_inter

In [14]:
aa_composition_ratio = []
for key in count_resi.keys():
    aa_composition_ratio.append(cdr_dict_big_cluster_aa_comp_res[key] / count_resi[key])

df_aa_comp = pd.DataFrame({'AA': cdr_dict_big_cluster_aa_comp_res.keys(),
    'Count': aa_composition_ratio})
    
fig_ring = px.histogram(df_aa_comp, x='AA', y='Count', 
    # color='color', barmode='group',
    height=400)
fig_ring.update_xaxes(title = "AA of the atom")
fig_ring.update_yaxes(title = "Probability", range = [0, 1])

### resname_of_the_carbons_from_the_biggest_hydrophobic_cluster_by_res_ratio_to_tot

In [15]:
aa_composition_ratio = []
for key in count_tot.keys():
    aa_composition_ratio.append(cdr_dict_big_cluster_aa_comp_res[key] / count_tot[key])

df_aa_comp = pd.DataFrame({'AA': cdr_dict_big_cluster_aa_comp_res.keys(),
    'Count': aa_composition_ratio})
    
fig_ring = px.histogram(df_aa_comp, x='AA', y='Count', 
    # color='color', barmode='group',
    height=400)
fig_ring.update_xaxes(title = "AA of the atom")
fig_ring.update_yaxes(title = "Probability", range = [0, .1])

### number_of_cdrs_in_the_biggest_cluster

In [16]:
figu = px.histogram(pd.DataFrame({'count':cdr_count_largest_cluster}),
    histnorm = 'probability', x='count', labels={'count': "# of CDRs in largest cluster"})
figu.update_xaxes(tick0 = 1, dtick=1)

---

## Ring

In [17]:
# There're some Pi-Pi interactions where one of the rings
# doesn't belong to a CDR. This doesn't happen with the hydrophobic interactions,
# since for those I just look at the CDR atoms.

with open(Path.joinpath(casa_dir, 'data', 'PiPi.pkl'), 'rb') as file:
    df_PiPi_atom_indices, df_PiPi_atom_serials,\
    df_PiPi_resSeq, df_PiPi_name, df_PiPi_chain_ID,\
    df_PiPi_chain_type, df_PiPi_cdr = pickle.load(file)

# There're some PiAnion and PiCation interactions where one ion/ring 
# doesn't belong to a CDR. This doesn't happen with the hydrophobic interactions,
# since for those I just look at the CDR atoms.

with open(Path.joinpath(casa_dir, 'data', 'PiAnion.pkl'), 'rb') as file:
    df_PiAnion_atom_indices, df_PiAnion_atom_serials,\
    df_PiAnion_resSeq, df_PiAnion_name, df_PiAnion_chain_ID,\
    df_PiAnion_chain_type, df_PiAnion_cdr = pickle.load(file)

with open(Path.joinpath(casa_dir, 'data', 'PiCation.pkl'), 'rb') as file:
    df_PiCation_atom_indices, df_PiCation_atom_serials,\
    df_PiCation_resSeq, df_PiCation_name, df_PiCation_chain_ID,\
    df_PiCation_chain_type, df_PiCation_cdr = pickle.load(file)    

In [18]:
pipi_dict = {'TYR': 0, 'PHE': 0, 'TRP': 0, 'HIS': 0}
pianion_dict = {'TYR': 0, 'PHE': 0, 'TRP': 0, 'HIS': 0}
pication_dict = {'TYR': 0, 'PHE': 0, 'TRP': 0, 'HIS': 0}

for pdb_idcode in pdb_list:
    
    for par in df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0]:
        pianion_dict[par.ring] += 1
    for par in df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0]:
        pication_dict[par.ring] += 1
    for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0]:
        pipi_dict[par.antigen] += 1
        pipi_dict[par.antibody] += 1

In [19]:
all_ring_interactions = \
[ pipi_dict['TYR'], pipi_dict['PHE'], pipi_dict['TRP'],
pianion_dict['TYR'], pianion_dict['PHE'], pianion_dict['TRP'], 
pication_dict['TYR'], pication_dict['PHE'], pication_dict['TRP'], ]

all_ring_group_interactions = list(itertools.repeat("Pi stacking", 3)) +\
    list(itertools.repeat("Pi anion", 3)) + list(itertools.repeat("Pi cation", 3))

all_ring_cdrs = list(itertools.chain.from_iterable(itertools.repeat(
    ['TYR', 'PHE', 'TRP'], 3)))

df_ring = pd.DataFrame({"y": all_ring_interactions, "x":all_ring_cdrs,
    "color":all_ring_group_interactions})

### ring_interactions

In [20]:
fig_ring = px.histogram(df_ring, x="x", y="y", 
             color='color', barmode='group',
             height=400)
fig_ring.update_xaxes(title = "")
fig_ring.update_yaxes(title = "Count", range=[0, 700])

-----

### Polar atoms

In [169]:
with open(Path.joinpath(casa_dir, 'data', 'shielding.pkl'), 'rb') as file:
    shielding_dict = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'hbonds_0_39.pkl'), 'rb') as file:
        hbonds = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_polar.pkl'), 'rb') as file:
        count_polar = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_hbonds.pkl'), 'rb') as file:
        polar_hbonds = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_saltBridge.pkl'), 'rb') as file:
        saltBridge = pickle.load(file)

In [170]:
polar_dict = {'hbond': 0, 'saltBridge': 0}

for pdb_idcode in pdb_list:
    polar_dict['saltBridge'] += len(saltBridge[pdb_idcode].keys()) / 2
    polar_dict['hbond'] += len(polar_hbonds[pdb_idcode].keys()) / 2

### polar_interactions

In [171]:
go.Figure(data = go.Pie(labels = ['Hydrogen bond', 'Salt bridge'], 
    values = [polar_dict['hbond'], polar_dict['saltBridge']], hole = .4) )

In [172]:
ON_count_dict_SC = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}
ON_count_dict_SC_all = {}

for pdb_idcode in pdb_list:
    shield_and_hbond = sum([ shielding_dict[pdb_idcode][id].is_sidechain\
        for id in set(shielding_dict[pdb_idcode].keys()).intersection(
        set(hbonds[pdb_idcode].keys()))])

    sh_pre = len(
        [ key for key, val in shielding_dict[pdb_idcode].items() if val.is_sidechain ])
    sh_cnt = sh_pre - shield_and_hbond
    
    # Hbonds are duplicated in this dict, hence every atom participating in hbond
    # shows up as a key. But I can't just count them, because some of them will be
    # backbone atoms, other sidechain or other O from waters.
    suma_h_pre = 0
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            suma_h_pre += int(Htuple.acceptor.is_sidechain) + int(Htuple.donor.is_sidechain)
    if suma_h_pre % 2:
        h_cnt = suma_h_pre // 2 + 1 - shield_and_hbond
    else:
        h_cnt = suma_h_pre // 2 - shield_and_hbond

    tot = count_polar[pdb_idcode].cdr_SC + count_polar[pdb_idcode].epi_SC

    if tot == 0:
        print(f"{pdb_idcode} has no polar contacts")
        continue
    
    # Counting proportions at the PDB-level, and not atom level. Hence, if 60% of
    # the polar atoms of a protein are shielding atoms, and that same value is 20% for
    # another protein, then the result will be that 40% of the polar atoms are shielding.
    # It doesn't matter if the first protein had a total of 20 polar atoms and the second
    # one had 200.
    ON_count_dict_SC['shields'] += sh_cnt / tot
    ON_count_dict_SC['hbonds'] += h_cnt / tot
    ON_count_dict_SC['shields_and_hbonds'] += shield_and_hbond / tot
    ON_count_dict_SC['none'] += (tot - sh_cnt - h_cnt - shield_and_hbond) / tot

    ON_count_pdb = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}
    ON_count_pdb['shields'] = sh_cnt / tot
    ON_count_pdb['hbonds'] = h_cnt / tot
    ON_count_pdb['shields_and_hbonds'] = shield_and_hbond / tot
    ON_count_pdb['none'] = (tot - sh_cnt - h_cnt - shield_and_hbond) / tot
    ON_count_dict_SC_all[pdb_idcode] = ON_count_pdb

### role_of_polar_atoms_SC_AB_AG

In [173]:
fig = go.Figure(data = go.Pie(labels = ['Shielding polar atom', 'Hydrogen bonding polar atom',
    'Shields and forms an Hbond', 'None'], 
    values = [ON_count_dict_SC['shields'], ON_count_dict_SC['hbonds'],
    ON_count_dict_SC['shields_and_hbonds'], ON_count_dict_SC['none']], hole = .4))
fig.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})

fig.update_layout(annotations=[dict(
    text='SC-AbAg', x=0.5, y=0.5, font_size=20, showarrow=False)])

In [174]:
ON_count_dict_BB = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}

for pdb_idcode in pdb_list:
    shield_and_hbond = sum([ not shielding_dict[pdb_idcode][id].is_sidechain\
        for id in set(shielding_dict[pdb_idcode].keys()).intersection(
        set(hbonds[pdb_idcode].keys()))])

    sh_pre = len(
        [ key for key, val in shielding_dict[pdb_idcode].items() if\
            not val.is_sidechain ])
    sh_cnt = sh_pre - shield_and_hbond

    # Hbonds are duplicated in this dict, hence every atom participating in hbond
    # shows up as a key. But I can't just count them, because some of them will be
    # backbone atoms, other sidechain or other O from waters.
    suma_h_pre = 0
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            suma_h_pre += int(not Htuple.acceptor.is_sidechain) +\
                int(not Htuple.donor.is_sidechain)
    if suma_h_pre % 2:
        h_cnt = suma_h_pre // 2 + 1 - shield_and_hbond
    else:
        h_cnt = suma_h_pre // 2 - shield_and_hbond
    
    tot = count_polar[pdb_idcode].cdr_BB + count_polar[pdb_idcode].epi_BB

    if tot == 0:
        print(f"{pdb_idcode} has no polar contacts")
        continue
    
    # Counting proportions at the PDB-level, and not atom level. Hence, if 60% of
    # the polar atoms of a protein are shielding atoms, and that same value is 20% for
    # another protein, then the result will be that 40% of the polar atoms are shielding.
    # It doesn't matter if the first protein had a total of 20 polar atoms and the second
    # one had 200.
    ON_count_dict_BB['shields'] += sh_cnt / tot
    ON_count_dict_BB['hbonds'] += h_cnt / tot
    ON_count_dict_BB['shields_and_hbonds'] += shield_and_hbond / tot
    ON_count_dict_BB['none'] += (tot - sh_cnt - h_cnt - shield_and_hbond) / tot

### role_of_polar_atoms_BB_AB_AG

In [175]:
fig = go.Figure(data = go.Pie(labels = ['Shielding polar atom', 'Hydrogen bonding polar atom',
    'Shields and forms an Hbond', 'None'], 
    values = [ON_count_dict_BB['shields'], ON_count_dict_BB['hbonds'],
    ON_count_dict_BB['shields_and_hbonds'], ON_count_dict_BB['none']], hole = .4))
fig.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})
fig.update_layout(annotations=[dict(
    text='BB-AbAg', x=0.5, y=0.5, font_size=20, showarrow=False)])

In [28]:
ON_count_dict_SC = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}

for pdb_idcode in pdb_list:
    shield_and_hbond = sum([ shielding_dict[pdb_idcode][id].is_sidechain and\
        (shielding_dict[pdb_idcode][id].chain_type != -1)\
        for id in set(shielding_dict[pdb_idcode].keys()).intersection(
        set(hbonds[pdb_idcode].keys()))])

    sh_pre = len(
        [ key for key, val in shielding_dict[pdb_idcode].items() if\
            val.is_sidechain and (val.chain_type != -1) ])
    sh_cnt = sh_pre - shield_and_hbond
    
    # Hbonds are duplicated in this dict, hence every atom participating in hbond
    # shows up as a key. But I can't just count them, because some of them will be
    # backbone atoms, other sidechain or other O from waters.
    suma_h_pre = 0
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            acc_is_ab = (Htuple.acceptor.chain_type != -1)
            don_is_ab = (Htuple.donor.chain_type != -1)
            suma_h_pre += int(Htuple.acceptor.is_sidechain and acc_is_ab) +\
                int(Htuple.donor.is_sidechain and don_is_ab)
    if suma_h_pre % 2:
        h_cnt = suma_h_pre // 2 + 1 - shield_and_hbond
    else:
        h_cnt = suma_h_pre // 2 - shield_and_hbond

    tot = count_polar[pdb_idcode].cdr_SC

    if tot == 0:
        print(f"{pdb_idcode} has no polar contacts")
        continue
    
    # Counting proportions at the PDB-level, and not atom level. Hence, if 60% of
    # the polar atoms of a protein are shielding atoms, and that same value is 20% for
    # another protein, then the result will be that 40% of the polar atoms are shielding.
    # It doesn't matter if the first protein had a total of 20 polar atoms and the second
    # one had 200.
    ON_count_dict_SC['shields'] += sh_cnt / tot
    ON_count_dict_SC['hbonds'] += h_cnt / tot
    ON_count_dict_SC['shields_and_hbonds'] += shield_and_hbond / tot
    ON_count_dict_SC['none'] += (tot - sh_cnt - h_cnt - shield_and_hbond) / tot

### role_of_polar_atoms_SC_AB

In [29]:
fig = go.Figure(data = go.Pie(labels = ['Shielding polar atom', 'Hydrogen bonding polar atom',
    'Shields and forms an Hbond', 'None'], 
    values = [ON_count_dict_SC['shields'], ON_count_dict_SC['hbonds'],
    ON_count_dict_SC['shields_and_hbonds'], ON_count_dict_SC['none']], hole = .4))
fig.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})
fig.update_layout(annotations=[dict(
    text='SC-Ab', x=0.5, y=0.5, font_size=20, showarrow=False)])

In [30]:
ON_count_dict_BB = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}

for pdb_idcode in pdb_list:
    shield_and_hbond = sum([ not shielding_dict[pdb_idcode][id].is_sidechain and\
        (shielding_dict[pdb_idcode][id].chain_type != -1)\
        for id in set(shielding_dict[pdb_idcode].keys()).intersection(
        set(hbonds[pdb_idcode].keys()))])

    sh_pre = len(
        [ key for key, val in shielding_dict[pdb_idcode].items() if\
            not val.is_sidechain and (val.chain_type != -1) ])
    sh_cnt = sh_pre - shield_and_hbond
    
    # Hbonds are duplicated in this dict, hence every atom participating in hbond
    # shows up as a key. But I can't just count them, because some of them will be
    # backbone atoms, other sidechain or other O from waters.
    suma_h_pre = 0
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            acc_is_ab = (Htuple.acceptor.chain_type != -1)
            don_is_ab = (Htuple.donor.chain_type != -1)
            suma_h_pre += int(not Htuple.acceptor.is_sidechain and acc_is_ab) +\
                int(not Htuple.donor.is_sidechain and don_is_ab)
    if suma_h_pre % 2:
        h_cnt = suma_h_pre // 2 + 1 - shield_and_hbond
    else:
        h_cnt = suma_h_pre // 2 - shield_and_hbond

    tot = count_polar[pdb_idcode].cdr_BB

    if tot == 0:
        print(f"{pdb_idcode} has no polar contacts")
        continue
    
    # Counting proportions at the PDB-level, and not atom level. Hence, if 60% of
    # the polar atoms of a protein are shielding atoms, and that same value is 20% for
    # another protein, then the result will be that 40% of the polar atoms are shielding.
    # It doesn't matter if the first protein had a total of 20 polar atoms and the second
    # one had 200.
    ON_count_dict_BB['shields'] += sh_cnt / tot
    ON_count_dict_BB['hbonds'] += h_cnt / tot
    ON_count_dict_BB['shields_and_hbonds'] += shield_and_hbond / tot
    ON_count_dict_BB['none'] += (tot - sh_cnt - h_cnt - shield_and_hbond) / tot

### role_of_polar_atoms_BB_AB

In [31]:
fig = go.Figure(data = go.Pie(labels = ['Shielding polar atom', 'Hydrogen bonding polar atom',
    'Shields and forms an Hbond', 'None'], 
    values = [ON_count_dict_BB['shields'], ON_count_dict_BB['hbonds'],
    ON_count_dict_BB['shields_and_hbonds'], ON_count_dict_BB['none']], hole = .4))
fig.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})
fig.update_layout(annotations=[dict(
    text='BB-Ab', x=0.5, y=0.5, font_size=20, showarrow=False)])

In [32]:
ON_count_dict_SC = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}

for pdb_idcode in pdb_list:
    shield_and_hbond = sum([ shielding_dict[pdb_idcode][id].is_sidechain and\
        (shielding_dict[pdb_idcode][id].chain_type == -1)\
        for id in set(shielding_dict[pdb_idcode].keys()).intersection(
        set(hbonds[pdb_idcode].keys()))])

    sh_pre = len(
        [ key for key, val in shielding_dict[pdb_idcode].items() if\
            val.is_sidechain and (val.chain_type == -1) ])
    sh_cnt = sh_pre - shield_and_hbond
    
    # Hbonds are duplicated in this dict, hence every atom participating in hbond
    # shows up as a key. But I can't just count them, because some of them will be
    # backbone atoms, other sidechain or other O from waters.
    suma_h_pre = 0
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            acc_is_ab = (Htuple.acceptor.chain_type == -1)
            don_is_ab = (Htuple.donor.chain_type == -1)
            suma_h_pre += int(Htuple.acceptor.is_sidechain and acc_is_ab) +\
                int(Htuple.donor.is_sidechain and don_is_ab)
    if suma_h_pre % 2:
        h_cnt = suma_h_pre // 2 + 1 - shield_and_hbond
    else:
        h_cnt = suma_h_pre // 2 - shield_and_hbond

    tot = count_polar[pdb_idcode].cdr_SC

    if tot == 0:
        print(f"{pdb_idcode} has no polar contacts")
        continue
    
    # Counting proportions at the PDB-level, and not atom level. Hence, if 60% of
    # the polar atoms of a protein are shielding atoms, and that same value is 20% for
    # another protein, then the result will be that 40% of the polar atoms are shielding.
    # It doesn't matter if the first protein had a total of 20 polar atoms and the second
    # one had 200.
    ON_count_dict_SC['shields'] += sh_cnt / tot
    ON_count_dict_SC['hbonds'] += h_cnt / tot
    ON_count_dict_SC['shields_and_hbonds'] += shield_and_hbond / tot
    ON_count_dict_SC['none'] += (tot - sh_cnt - h_cnt - shield_and_hbond) / tot

### role_of_polar_atoms_SC_AG

In [33]:
fig = go.Figure(data = go.Pie(labels = ['Shielding polar atom', 'Hydrogen bonding polar atom',
    'Shields and forms an Hbond', 'None'], 
    values = [ON_count_dict_SC['shields'], ON_count_dict_SC['hbonds'],
    ON_count_dict_SC['shields_and_hbonds'], ON_count_dict_SC['none']], hole = .4))
fig.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})
fig.update_layout(annotations=[dict(
    text='SC-Ag', x=0.5, y=0.5, font_size=20, showarrow=False)])

In [34]:
ON_count_dict_BB = {'shields': 0, 'hbonds': 0, 'shields_and_hbonds': 0, 'none': 0}

for pdb_idcode in pdb_list:
    shield_and_hbond = sum([ not shielding_dict[pdb_idcode][id].is_sidechain and\
        (shielding_dict[pdb_idcode][id].chain_type == -1)\
        for id in set(shielding_dict[pdb_idcode].keys()).intersection(
        set(hbonds[pdb_idcode].keys()))])

    sh_pre = len(
        [ key for key, val in shielding_dict[pdb_idcode].items() if\
            not val.is_sidechain and (val.chain_type == -1) ])
    sh_cnt = sh_pre - shield_and_hbond
    
    # Hbonds are duplicated in this dict, hence every atom participating in hbond
    # shows up as a key. But I can't just count them, because some of them will be
    # backbone atoms, other sidechain or other O from waters.
    suma_h_pre = 0
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            acc_is_ab = (Htuple.acceptor.chain_type == -1)
            don_is_ab = (Htuple.donor.chain_type == -1)
            suma_h_pre += int(not Htuple.acceptor.is_sidechain and acc_is_ab) +\
                int(not Htuple.donor.is_sidechain and don_is_ab)
    if suma_h_pre % 2:
        h_cnt = suma_h_pre // 2 + 1 - shield_and_hbond
    else:
        h_cnt = suma_h_pre // 2 - shield_and_hbond

    tot = count_polar[pdb_idcode].cdr_BB

    if tot == 0:
        print(f"{pdb_idcode} has no polar contacts")
        continue
    
    # Counting proportions at the PDB-level, and not atom level. Hence, if 60% of
    # the polar atoms of a protein are shielding atoms, and that same value is 20% for
    # another protein, then the result will be that 40% of the polar atoms are shielding.
    # It doesn't matter if the first protein had a total of 20 polar atoms and the second
    # one had 200.
    ON_count_dict_BB['shields'] += sh_cnt / tot
    ON_count_dict_BB['hbonds'] += h_cnt / tot
    ON_count_dict_BB['shields_and_hbonds'] += shield_and_hbond / tot
    ON_count_dict_BB['none'] += (tot - sh_cnt - h_cnt - shield_and_hbond) / tot

### role_of_polar_atoms_BB_AG

In [35]:
fig = go.Figure(data = go.Pie(labels = ['Shielding polar atom', 'Hydrogen bonding polar atom',
    'Shields and forms an Hbond', 'None'], 
    values = [ON_count_dict_BB['shields'], ON_count_dict_BB['hbonds'],
    ON_count_dict_BB['shields_and_hbonds'], ON_count_dict_BB['none']], hole = .4))
fig.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})
fig.update_layout(annotations=[dict(
    text='BB-Ag', x=0.5, y=0.5, font_size=20, showarrow=False)])

---

## DSSP

In [106]:
with open(Path.joinpath(casa_dir, 'data', 'simple_SSE.pkl'), 'rb') as file:
    SSE = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'simple_SSE_count.pkl'), 'rb') as file:
    SSE_cnt = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'simple_SSE_cnt_interface.pkl'), 'rb') as file:
    SSE_cnt_interface = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)

In [107]:
sse_epi_clu_atm = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_clu_res = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_clu_big_atm = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_clu_big_res = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_all = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_interface = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_ratio_clu_atm = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_ratio_clu_res = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_ratio_clu_big_atm = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_epi_ratio_clu_big_res = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
avg_ratio_res_atm = np.array([])

# check_pdb = '1afv'
# idx = pdb_list.index(check_pdb)
# for pdb_idcode in [pdb_list[idx]]:    
for pdb_idcode in pdb_list:
    chain_type_of_each_contact = (df_hydrophobic_chain_type.query(
        f"idcode == '{pdb_idcode}'").chain_type)[0]
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    cdr_of_each_contact = (df_hydrophobic_cdr.query(
        f"idcode == '{pdb_idcode}'").CDR)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname[0])
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq[0])

    # First, just count the SSE of all antigen residues and of those at the interface
    for sse in SSE_cnt[pdb_idcode].keys():
        sse_epi_all[sse] += SSE_cnt[pdb_idcode][sse]
        sse_epi_interface[sse] += SSE_cnt_interface[pdb_idcode][sse]

    flag = True
    sse_epi_clu_atm_pdb = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
    sse_epi_clu_res_pdb = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
    sse_epi_clu_big_atm_pdb = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
    sse_epi_clu_big_res_pdb = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
    for cluster_cdr, cluster_chainID, cluster_chain_type,\
        cluster_resname, cluster_resSeq in\
        zip(cdr_of_each_contact, chainID_of_each_contact,
        chain_type_of_each_contact, resname_of_each_contact, resSeq_of_each_contact):
        
        # Now I have to make sure I'm counting residues instead of atoms.
        sse_list_resname_resSeq_chainID = []
        for resname, resSeq, chainID in zip(cluster_resname, cluster_resSeq, cluster_chainID):
            if chainID in chains[pdb_idcode].antigen:
                sse_list_resname_resSeq_chainID.append(resname + chainID + str(resSeq))


        unique_resi_list = set(sse_list_resname_resSeq_chainID)
        # get the ResSeq of each unique residue
        unique_resSeq = [ int(unique_[4:]) for unique_ in unique_resi_list ]
        for unique_ in unique_resSeq:
            sse = SSE[pdb_idcode][unique_]
            # Residue level count.
            sse_epi_clu_res[sse] += 1
            sse_epi_clu_res_pdb[sse] += 1

        # Now, count atoms
        for contact_cdr, contact_chain_type, contact_resname, contact_resSeq in\
            zip(cluster_cdr, cluster_chain_type, cluster_resname,
            cluster_resSeq):

            if contact_chain_type == '':
                # antigen carbons
                sse = SSE[pdb_idcode][contact_resSeq]
                # Atomic level count
                sse_epi_clu_atm[sse] += 1
                sse_epi_clu_atm_pdb[sse] += 1
        # Biggest cluster
        if flag:
            flag = False
            # Count SSEs in the largest cluster
            for contact_chain_type, contact_resSeq in zip(cluster_chain_type, cluster_resSeq):
                if contact_chain_type == '':
                    # antigen carbons
                    sse = SSE[pdb_idcode][contact_resSeq]
                    # Atomic level count
                    sse_epi_clu_big_atm[sse] += 1
                    sse_epi_clu_big_atm_pdb[sse] += 1
            # Now I have to make sure I'm counting residues instead of atoms.
            for unique_ in unique_resSeq:
                sse = SSE[pdb_idcode][unique_]
                # Residue level count.
                sse_epi_clu_big_res[sse] += 1
                sse_epi_clu_big_res_pdb[sse] += 1
        # Finally, get the ratio of atom/residue
        avg_ratio_res_atm = np.append(avg_ratio_res_atm, len(resi_list) / len(unique_resi_list))

    # Ratios for all clusters
    for sse in sse_epi_clu_atm_pdb.keys():
        if sse_epi_clu_atm_pdb[sse] != 0:
            sse_epi_ratio_clu_atm[sse] += sse_epi_clu_atm_pdb[sse] / SSE_cnt[pdb_idcode][sse]
    for sse in sse_epi_clu_res_pdb.keys():
        if sse_epi_clu_res_pdb[sse] != 0:
            sse_epi_ratio_clu_res[sse] += sse_epi_clu_res_pdb[sse] / SSE_cnt[pdb_idcode][sse]
    # Ratios for the biggest cluster of each PDB
    for sse in sse_epi_clu_big_atm_pdb.keys():
        if sse_epi_clu_big_atm_pdb[sse] != 0:
            sse_epi_ratio_clu_big_atm[sse] += sse_epi_clu_big_atm_pdb[sse] / SSE_cnt[pdb_idcode][sse]
    for sse in sse_epi_clu_big_res_pdb.keys():
        if sse_epi_clu_big_res_pdb[sse] != 0:
            sse_epi_ratio_clu_big_res[sse] += sse_epi_clu_big_res_pdb[sse] / SSE_cnt[pdb_idcode][sse]

In [108]:
del sse_epi_ratio_clu_atm['NA']
for sse, val in sse_epi_ratio_clu_atm.items():
    sse_epi_ratio_clu_atm[sse] = val / len(pdb_list)

del sse_epi_ratio_clu_res['NA']
for sse, val in sse_epi_ratio_clu_res.items():
    sse_epi_ratio_clu_res[sse] = val / len(pdb_list)

del sse_epi_all['NA']
tot = sum(sse_epi_all.values())
sse_epi_all_ratio = {}
for sse, val in sse_epi_all.items():
    sse_epi_all_ratio[sse] = val / tot

del sse_epi_interface['NA']
tot = sum(sse_epi_interface.values())
sse_epi_interface_ratio = {}
for sse, val in sse_epi_interface.items():
    sse_epi_interface_ratio[sse] = val / tot
    
del sse_epi_clu_res['NA']
del sse_epi_clu_atm['NA']
np.mean(avg_ratio_res_atm)

13.181966070290256

### DSSP_of_antigen

In [136]:
df_sse_epi = pd.DataFrame({'sse': list(sse_epi_all_ratio.keys()) + list(sse_epi_interface_ratio.keys()), 
    'probability': list(sse_epi_all_ratio.values()) + list(sse_epi_interface_ratio.values()) ,
    'selection': np.repeat(['whole antigen','antigen interface'], 3)})
    
px.bar(df_sse_epi, x='sse', y='probability', color='selection', barmode='group')

### DSSP_of_all_antigen_residues

In [102]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_all['H'], sse_epi_all['C'], sse_epi_all['E']], hole = .4))
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

In [109]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_interface['H'], sse_epi_interface['C'], sse_epi_interface['E']], hole = .4))
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

### DSSP_of_carbons_in_hydrophobic_interactions_atm

In [40]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_clu_atm['H'], sse_epi_clu_atm['C'], sse_epi_clu_atm['E']], hole = .4))
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

### DSSP_of_carbons_in_hydrophobic_interactions_res

In [41]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_clu_res['H'], sse_epi_clu_res['C'], sse_epi_clu_res['E']], hole = .4))
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

### DSSP_of_carbons_from_biggest_hydrophobic_cluster_atm

In [42]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_clu_big_atm['H'], sse_epi_clu_big_atm['C'],
        sse_epi_clu_big_atm['E']], hole = .4))
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

### DSSP_of_carbons_from_biggest_hydrophobic_cluster_res

In [43]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_clu_big_res['H'], sse_epi_clu_big_res['C'],
        sse_epi_clu_big_res['E']], hole = .4))
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

In [44]:
chain_type_hbond_dict = {'H': 0, 'K': 0, 'L': 0}
cdr_hbond_dict = {'H1': 0, 'H2': 0, 'H3': 0, 'L1': 0, 'L2': 0, 'L3': 0, 
'K1': 0, 'K2': 0, 'K3': 0, 'H0': 0, 'K0': 0, 'L0': 0}
sse_epi_hbo = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
sse_all_hbo = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}

for pdb_idcode in pdb_list:
    for atm_id, bond_list in hbonds[pdb_idcode].items():
        for Htuple in bond_list:
            for Hatm in Htuple:
                if Hatm.chain_type != -1:
                    # Antibody residue.
                    chain_type_hbond_dict[Hatm.chain_type] += 1
                    cdr = Hatm.chain_type + str(Hatm.CDR)
                    cdr_hbond_dict[cdr] += 1
                elif Hatm.resname != 'HOH':
                    # Antigen residue.
                    # Atomic count.
                    sse = SSE[pdb_idcode][Hatm.resSeq]
                    sse_epi_hbo[sse] += 1

### DSSP_of_hbonding_atoms_residues_atm

In [45]:
fig = go.Figure(data = go.Pie(labels = ['Helix', 'Coil', 'β strand'], 
    values = [sse_epi_hbo['H'], sse_epi_hbo['C'], sse_epi_hbo['E']], hole = .4) )
fig.update_traces(marker={'colors': ['red', 'green', 'RoyalBLue']})

### percentage_of_residues_from_each_SSE_with_carbons_in_clusters_atm

In [46]:
figu = px.bar(x=sse_epi_ratio_clu_atm.keys(), y=sse_epi_ratio_clu_atm.values(),
labels={'x': " ", 'y': "Count"})
figu.update_yaxes(title = "Probability", range=[0, 0.3])
figu.show()

### percentage_of_residues_from_each_SSE_with_carbons_in_clusters_res

In [47]:
figu = px.bar(x=sse_epi_ratio_clu_res.keys(), y=sse_epi_ratio_clu_res.values(),
labels={'x': " ", 'y': "Count"})
figu.update_yaxes(title = "Probability", range=[0, 0.3])
figu.show()

---

### more hydrophobic

In [48]:
combinations_list = []
combinations_tuples = []
for pdb_idcode in pdb_list:
    chain_type_of_each_contact = (df_hydrophobic_chain_type.query(
        f"idcode == '{pdb_idcode}'").chain_type)[0]
    cdr_of_each_contact = (df_hydrophobic_cdr.query(
        f"idcode == '{pdb_idcode}'").CDR)[0]
    cluster_cdr = cdr_of_each_contact[0]
    cluster_chain_type = chain_type_of_each_contact[0]
    cdrs_in_big_cluster = []
    for chain, cdr in zip (cluster_chain_type, cluster_cdr):
        if cdr != -1:
            if chain == 'H':
                cdrs_in_big_cluster.append(chain+str(cdr))
            else:
                # 'K' and 'L' are the same
                cdrs_in_big_cluster.append('L'+str(cdr))

    combinations_list.append(list(filter(lambda x: x!= -1, Counter(cdrs_in_big_cluster).keys())))
    combinations_tuples.append(tuple(filter(lambda x: x!= -1, Counter(cdrs_in_big_cluster).keys())))

In [49]:
combi_dict = {}
combi_keys = []
combi_vals = []

for key, val in Counter(combinations_tuples).most_common():
    if len(key) != 1:
        combi_dict[key] = val
        combi_keys.append(' '.join(key))
        combi_vals.append(val)

px.bar(x=combi_keys[0:20], y= combi_vals[0:20])

In [50]:
cdr_idx = {'H1': 0, 'H2': 1, 'H3': 2, 'L1': 3, 'L2': 4, 'L3': 5}
cdrs = ['H1', 'H2', 'H3', 'L1', 'L2', 'L3']

cdr_pairs_lst_2 = [ '-'.join((a, b)) for a, b in itertools.combinations(cdrs, 2) ]
cdr_pairs_dct_2 = dict(zip(cdr_pairs_lst_2, itertools.repeat(0, math.comb(6, 2))))

cdr_pairs_lst_3 = [ '-'.join((a, b, c)) for a, b, c in itertools.combinations(cdrs, 3) ]
cdr_pairs_dct_3 = dict(zip(cdr_pairs_lst_3, itertools.repeat(0, math.comb(6, 3))))

cdr_pairs_lst_4 = [ '-'.join((a, b, c, d)) for a, b, c, d in itertools.combinations(cdrs, 4) ]
cdr_pairs_dct_4 = dict(zip(cdr_pairs_lst_4, itertools.repeat(0, math.comb(6, 4))))

In [51]:
for cdr_a, cdr_b in list(itertools.combinations(cdrs, 2)):
    for combi in combinations_tuples:
        if cdr_a in combi and cdr_b in combi:
            key = '-'.join((cdr_a, cdr_b))
            cdr_pairs_dct_2[key] += 1

In [52]:
for cdr_a, cdr_b, cdr_c in list(itertools.combinations(cdrs, 3)):
    for combi in combinations_tuples:
        if cdr_a in combi and cdr_b in combi and cdr_c in combi:
            key = '-'.join((cdr_a, cdr_b, cdr_c))
            cdr_pairs_dct_3[key] += 1

In [53]:
for cdr_a, cdr_b, cdr_c, cdr_d in list(itertools.combinations(cdrs, 4)):
    for combi in combinations_tuples:
        if cdr_a in combi and cdr_b in combi and cdr_c in combi and cdr_d in combi:
            key = '-'.join((cdr_a, cdr_b, cdr_c, cdr_d))
            cdr_pairs_dct_4[key] += 1

### cdrs_combinations_in_biggest_cluster_2

In [54]:
go.Figure(data = go.Pie(labels = list(cdr_pairs_dct_2.keys()), 
    values = list(cdr_pairs_dct_2.values()), hole = .4))

### cdrs_combinations_in_biggest_cluster_3

In [55]:
go.Figure(data = go.Pie(labels = list(cdr_pairs_dct_3.keys()), 
    values = list(cdr_pairs_dct_3.values()), hole = .4))

### cdrs_combinations_in_biggest_cluster_4

In [56]:
go.Figure(data = go.Pie(labels = list(cdr_pairs_dct_4.keys()), 
    values = list(cdr_pairs_dct_4.values()), hole = .4))

In [57]:
first_over_2nd_res = []
first_over_2nd_atm = []

for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    cdr_of_each_contact = (df_hydrophobic_cdr.query(
        f"idcode == '{pdb_idcode}'").CDR)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    
    cluster_chainID_0 = chainID_of_each_contact[0]
    cluster_cdr_0 = cdr_of_each_contact[0]
    cluster_resSeq_0 = resSeq_of_each_contact[0]
    
    if len(cluster_chainID_0) < 5 or len(chainID_of_each_contact) == 1:
        continue
    
    cluster_chainID_1 = chainID_of_each_contact[1]
    cluster_cdr_1 = cdr_of_each_contact[1]
    cluster_resSeq_1 = resSeq_of_each_contact[1]

    # Now I have to make sure I'm counting residues, instead of atoms.
    resi_list_0 = []
    resi_list_1 = []
    # using both resSeq and chainID to identify each residue,
    # since there're chainID overlaps
    for chain, res in zip (cluster_chainID_0, cluster_resSeq_0):
        resi_list_0.append(chain+str(res))
    for chain, res in zip (cluster_chainID_1, cluster_resSeq_1):
        resi_list_1.append(chain+str(res))

    cnt_res_0 = len(set(resi_list_0))
    cnt_res_1 = len(set(resi_list_1))
    first_over_2nd_res.append(cnt_res_1/cnt_res_0)
    cnt_atm_0 = len(cluster_chainID_0)
    cnt_atm_1 = len(cluster_chainID_1)
    first_over_2nd_atm.append(cnt_atm_1/cnt_atm_0)

### second_biggest_cluster_over_biggest_atm

In [58]:
figu = px.histogram(pd.DataFrame({'count':first_over_2nd_atm}), nbins = 10,
    histnorm = 'probability', x='count', labels={'count': "# of clusters"})
figu.update_xaxes(tick0 = 1, dtick=.1)
figu.update_layout(bargap = .01)
figu.update_xaxes(title = "2nd largest cluster / largest cluster")
figu.show()

### second_biggest_cluster_over_biggest_res

In [59]:
figu = px.histogram(pd.DataFrame({'count':first_over_2nd_res}), nbins = 10,
    histnorm = 'probability', x='count', labels={'count': "# of clusters"})
figu.update_xaxes(tick0 = 1, dtick=.2)
figu.update_layout(bargap = .01)
figu.update_xaxes(title = "2nd largest cluster / largest cluster")
figu.show()

---

### more ring interactions

In [60]:
with open(Path.joinpath(casa_dir, 'data', 'count_tyrs.pkl'), 'rb') as file:
    count_tyrs = pickle.load(file)
# There're some Pi-Pi interactions where one of the rings
# doesn't belong to a CDR. This doesn't happen with the hydrophobic interactions,
# since for those I just look at the CDR atoms.

with open(Path.joinpath(casa_dir, 'data', 'PiPi.pkl'), 'rb') as file:
    df_PiPi_atom_indices, df_PiPi_atom_serials,\
    df_PiPi_resSeq, df_PiPi_name, df_PiPi_chain_ID,\
    df_PiPi_chain_type, df_PiPi_cdr = pickle.load(file)

# There're some PiAnion and PiCation interactions where one ion/ring 
# doesn't belong to a CDR. This doesn't happen with the hydrophobic interactions,
# since for those I just look at the CDR atoms.

with open(Path.joinpath(casa_dir, 'data', 'PiAnion.pkl'), 'rb') as file:
    df_PiAnion_atom_indices, df_PiAnion_atom_serials,\
    df_PiAnion_resSeq, df_PiAnion_name, df_PiAnion_chain_ID,\
    df_PiAnion_chain_type, df_PiAnion_cdr = pickle.load(file)

with open(Path.joinpath(casa_dir, 'data', 'PiCation.pkl'), 'rb') as file:
    df_PiCation_atom_indices, df_PiCation_atom_serials,\
    df_PiCation_resSeq, df_PiCation_name, df_PiCation_chain_ID,\
    df_PiCation_chain_type, df_PiCation_cdr = pickle.load(file)

# This dictionary has the total number of TYRs in interfaces
with open(Path.joinpath(casa_dir, 'data', 'count_resi.pkl'), 'rb') as file:
    count_resi = pickle.load(file)

# This dictionary has the total number of TYRs in interfaces, for each molecule of
# each PDB
with open(Path.joinpath(casa_dir, 'data', 'count_resi_pdb.pkl'), 'rb') as file:
    count_resi_pdb = pickle.load(file)

with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)

In [61]:
tyr_interactions = {'Hydrophobic': 0, 'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}


for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname)[0]
    
    ##### 
    # Hydrophobic
    #####
    # Now I have to make sure I'm counting tyrosines, instead of their atoms.
    list_resSeq = [ resSeq for cluster_resSeq in resSeq_of_each_contact\
        for resSeq in cluster_resSeq ]
    list_resname = [ resname for cluster_resname in resname_of_each_contact\
        for resname in cluster_resname ]
    list_chainID = [ chainID for cluster_chainID in chainID_of_each_contact\
        for chainID in cluster_chainID ]

    list_resSeq_chainID = []
    for resSeq, chainID in zip(list_resSeq, list_chainID):
        list_resSeq_chainID.append(str(resSeq) + chainID)

    unique_residues = set(list_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[:-1]))\
        for unique_ in unique_residues ]
    tyr_hydro_cnt = sum([ list_resname[i] == 'TYR' for i in unique_indices ])

    ##### 
    # Ring
    #####
    tyr_Anion_cnt = sum([ par.ring == 'TYR' \
        for par in df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    tyr_Cation_cnt = sum([ par.ring == 'TYR' \
        for par in df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    tyr_PiPi_cnt = sum([ par.antibody == 'TYR' \
        for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0] ]) +\
    sum([ par.antigen == 'TYR' \
        for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])

    ##### 
    # Hbond
    #####
    list_resSeq = []
    list_resname = []
    list_chainID = []
    list_wat_resSeq = []
    list_wat_resname = []
    list_wat_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
            if Htuple.acceptor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.donor.resSeq)
                list_wat_resname.append(Htuple.donor.resname)
                list_wat_chainID.append(Htuple.donor.chainID)
            elif Htuple.donor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.acceptor.resSeq)
                list_wat_resname.append(Htuple.acceptor.resname)
                list_wat_chainID.append(Htuple.acceptor.chainID)
            else:
                list_resSeq.append(Htuple.acceptor.resSeq)
                list_resname.append(Htuple.acceptor.resname)
                list_chainID.append(Htuple.acceptor.chainID)
                list_resSeq.append(Htuple.donor.resSeq)
                list_resname.append(Htuple.donor.resname)
                list_chainID.append(Htuple.donor.chainID)

    # No WAT
    list_resSeq_chainID = []
    for resSeq, chainID in zip(list_resSeq, list_chainID):
        list_resSeq_chainID.append(str(resSeq) + chainID)

    unique_residues = set(list_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[:-1]))\
        for unique_ in unique_residues ]

    tyr_hbond_cnt = sum([ list_resname[i] == 'TYR' for i in unique_indices ])
    # WAT
    hbond_list_wat_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_wat_resname, list_wat_resSeq, list_wat_chainID):
        hbond_list_wat_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(hbond_list_wat_resname_resSeq_chainID)
    unique_indices = [ list_wat_resSeq.index(int(unique_[4:]))\
        for unique_ in unique_residues ]

    tyr_hbond_wat_cnt = sum([ list_wat_resname[i] == 'TYR' for i in unique_indices ])
    
    ##### 
    # Total
    #####
    tot_cnt = count_tyrs[pdb_idcode].heavy + count_tyrs[pdb_idcode].light\
        + count_tyrs[pdb_idcode].antigen
    tyr_interactions['PiAnion'] += tyr_Anion_cnt
    tyr_interactions['PiCation'] += tyr_Cation_cnt
    tyr_interactions['PiPi'] += tyr_PiPi_cnt
    tyr_interactions['Hydrophobic'] += tyr_hydro_cnt
    tyr_interactions['Water mediated'] += tyr_hbond_wat_cnt
    tyr_interactions['Hbond'] += tyr_hbond_cnt
    tyr_interactions['Total'] += tot_cnt

tyr_interactions_ratio_tot = {}
for key, val in tyr_interactions.items():
    tyr_interactions_ratio_tot[key] = val / tyr_interactions['Total']

tyr_interactions_ratio_inter = {}
for key, val in tyr_interactions.items():
    tyr_interactions_ratio_inter[key] = val / count_resi['TYR']

### tyr_interactions_ratio_tot

In [62]:
df_tyr = pd.DataFrame({"y": list(tyr_interactions_ratio_tot.values())[:-1],
    "x":list(tyr_interactions_ratio_tot.keys())[:-1]})
    
fig_tyr = px.bar(df_tyr, x="x", y="y")
fig_tyr.update_xaxes(title = "")
fig_tyr.update_yaxes(title = "Interacting # of TYRs / Total # of TYRs", range=[0, .2])

### tyr_interactions_ratio_inter

In [63]:
df_tyr = pd.DataFrame({"y": list(tyr_interactions_ratio_inter.values())[:-1],
    "x":list(tyr_interactions_ratio_inter.keys())[:-1]})
    
fig_tyr = px.bar(df_tyr, x="x", y="y")
fig_tyr.update_xaxes(title = "")
fig_tyr.update_yaxes(title = "# of interacting TYRs / # of interface TYRs", range=[0, 1])

In [64]:
tyr_interactions_ab = {'Hydrophobic': 0, 'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}
tyr_interactions_ag = {'Hydrophobic': 0, 'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}

for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname)[0]
    
    ##### 
    # Hydrophobic
    #####
    # Now I have to make sure I'm counting tyrosines, instead of their atoms.
    hydro_list_name_Seq_ID_ab = []
    hydro_list_name_Seq_ID_ag = []
    for cluster_resSeq, cluster_resname, cluster_chainID in\
        zip(resSeq_of_each_contact, resname_of_each_contact, chainID_of_each_contact):
        for resSeq, resname, chainID in\
            zip(cluster_resSeq, cluster_resname, cluster_chainID):
            if resname != 'TYR':
                continue
            if chainID in chains[pdb_idcode].antibody:
                hydro_list_name_Seq_ID_ab.append(resname + chainID + str(resSeq))
            elif chainID in chains[pdb_idcode].antigen:
                hydro_list_name_Seq_ID_ag.append(resname + chainID + str(resSeq))
            else:
                logging.warning(f"This shouldn't happen")
    
    tyr_hydro_cnt_ab = len(set(hydro_list_name_Seq_ID_ab))
    tyr_hydro_cnt_ag = len(set(hydro_list_name_Seq_ID_ag))
    
    ##### 
    # Ring
    #####
    # Anion Ab
    Anion_list_name_Seq_ID_ab = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiAnion_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiAnion_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antibody]
    tyr_Anion_cnt_ab = sum([ each[0:3] == 'TYR' and each[3] in chains[pdb_idcode].antibody\
        for each in Anion_list_name_Seq_ID_ab])
    # Anion Ag
    Anion_list_name_Seq_ID_ag = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiAnion_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiAnion_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antigen]
    tyr_Anion_cnt_ag = sum([ each[0:3] == 'TYR' and each[3] in chains[pdb_idcode].antigen\
        for each in Anion_list_name_Seq_ID_ag])

    # Cation Ab
    Cation_list_name_Seq_ID_ab = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiCation_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiCation_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antibody]
    tyr_Cation_cnt_ab = sum([ each[0:3] == 'TYR' and each[3] in chains[pdb_idcode].antibody\
        for each in Cation_list_name_Seq_ID_ab])
    # Cation Ag
    Cation_list_name_Seq_ID_ag = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiCation_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiCation_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antigen]
    tyr_Cation_cnt_ag = sum([ each[0:3] == 'TYR' and each[3] in chains[pdb_idcode].antigen\
        for each in Cation_list_name_Seq_ID_ag])

    # Pi Ab
    Pi_list_name_Seq_ID_ab = [ pair_resname.antibody + pair_chainID.antibody + str(pair_resSeq.antibody)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiPi_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiPi_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.antibody == 'TYR' and pair_chainID.antibody in chains[pdb_idcode].antibody]
    tyr_Pi_cnt_ab = sum([ each[0:3] == 'TYR' and each[3] in chains[pdb_idcode].antibody\
        for each in Pi_list_name_Seq_ID_ab])
    # Pi Ag
    Pi_list_name_Seq_ID_ag = [ pair_resname.antigen + pair_chainID.antigen + str(pair_resSeq.antigen)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiPi_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiPi_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.antigen == 'TYR' and pair_chainID.antigen in chains[pdb_idcode].antigen]
    tyr_Pi_cnt_ag = sum([ each[0:3] == 'TYR' and each[3] in chains[pdb_idcode].antigen\
        for each in Pi_list_name_Seq_ID_ag])

    
    ##### 
    # Hbond
    #####
    list_resSeq = []
    list_resname = []
    list_chainID = []
    list_wat_resSeq = []
    list_wat_resname = []
    list_wat_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
            if Htuple.acceptor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.donor.resSeq)
                list_wat_resname.append(Htuple.donor.resname)
                list_wat_chainID.append(Htuple.donor.chainID)
            elif Htuple.donor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.acceptor.resSeq)
                list_wat_resname.append(Htuple.acceptor.resname)
                list_wat_chainID.append(Htuple.acceptor.chainID)
            else:
                list_resSeq.append(Htuple.acceptor.resSeq)
                list_resname.append(Htuple.acceptor.resname)
                list_chainID.append(Htuple.acceptor.chainID)
                list_resSeq.append(Htuple.donor.resSeq)
                list_resname.append(Htuple.donor.resname)
                list_chainID.append(Htuple.donor.chainID)

    # No WAT
    hbond_list_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_resname, list_resSeq, list_chainID):
        hbond_list_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(hbond_list_resname_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[4:]))\
        for unique_ in unique_residues ]

    tyr_hbond_cnt_ab = sum([ list_resname[i] == 'TYR' and list_chainID[i] in chains[pdb_idcode].antibody\
        for i in unique_indices ])
    tyr_hbond_cnt_ag = sum([ list_resname[i] == 'TYR' and list_chainID[i] in chains[pdb_idcode].antigen\
        for i in unique_indices ])

    # WAT
    hbond_list_wat_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_wat_resname, list_wat_resSeq, list_wat_chainID):
        hbond_list_wat_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(hbond_list_wat_resname_resSeq_chainID)
    unique_indices = [ list_wat_resSeq.index(int(unique_[4:]))\
        for unique_ in unique_residues ]

    tyr_hbond_wat_cnt_ab = sum([ list_wat_resname[i] == 'TYR' and list_wat_chainID[i] in chains[pdb_idcode].antibody\
        for i in unique_indices ])
    tyr_hbond_wat_cnt_ag = sum([ list_wat_resname[i] == 'TYR' and list_wat_chainID[i] in chains[pdb_idcode].antigen\
        for i in unique_indices ])

    ##### 
    # Total
    #####
    tyr_interactions_ab['Hydrophobic'] += tyr_hydro_cnt_ab
    tyr_interactions_ab['PiAnion'] += tyr_Anion_cnt_ab
    tyr_interactions_ab['PiCation'] += tyr_Cation_cnt_ab
    tyr_interactions_ab['PiPi'] += tyr_Pi_cnt_ab
    tyr_interactions_ab['Hbond'] += tyr_hbond_cnt_ab
    tyr_interactions_ab['Water mediated'] += tyr_hbond_wat_cnt_ab
    tyr_interactions_ab['Total'] += count_resi_pdb[pdb_idcode].antibody['TYR']
    tyr_interactions_ag['Hydrophobic'] += tyr_hydro_cnt_ag
    tyr_interactions_ag['PiAnion'] += tyr_Anion_cnt_ag
    tyr_interactions_ag['PiCation'] += tyr_Cation_cnt_ag
    tyr_interactions_ag['PiPi'] += tyr_Pi_cnt_ag
    tyr_interactions_ag['Hbond'] += tyr_hbond_cnt_ag
    tyr_interactions_ag['Water mediated'] += tyr_hbond_wat_cnt_ag
    tyr_interactions_ag['Total'] += count_resi_pdb[pdb_idcode].antigen['TYR']

tyr_interactions_ratio_inter_ab = {}
for key, val in tyr_interactions_ab.items():
    tyr_interactions_ratio_inter_ab[key] = val / tyr_interactions_ab['Total']
tyr_interactions_ratio_inter_ag = {}
for key, val in tyr_interactions_ag.items():
    tyr_interactions_ratio_inter_ag[key] = val / tyr_interactions_ag['Total']

### tyr_interactions_ratio_inter_ab

In [65]:
df_tyr = pd.DataFrame({"y": list(tyr_interactions_ratio_inter_ab.values())[:-1],
    "x":list(tyr_interactions_ratio_inter_ab.keys())[:-1]})

fig_tyr = px.bar(df_tyr, x="x", y="y", title="antibody")
fig_tyr.update_xaxes(title = "")
fig_tyr.update_yaxes(title = "# of interacting TYRs / # of interface TYRs", range=[0, 1])

### tyr_interactions_ratio_inter_ag

In [66]:
df_tyr = pd.DataFrame({"y": list(tyr_interactions_ratio_inter_ag.values())[:-1],
    "x":list(tyr_interactions_ratio_inter_ag.keys())[:-1]})
    
fig_tyr = px.bar(df_tyr, x="x", y="y", title="antigen")
fig_tyr.update_xaxes(title = "")
fig_tyr.update_yaxes(title = "# of interacting TYRs / # of interface TYRs", range=[0, 1])

In [67]:
tyr_combis_hydro_ab = {'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}
tyr_combis_hydro_ag = {'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}

for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname)[0]
    
    ##### 
    # Hydrophobic
    #####
    # Now I have to make sure I'm counting tyrosines, instead of their atoms.
    hydro_list_name_Seq_ID_ab = []
    hydro_list_name_Seq_ID_ag = []
    for cluster_resSeq, cluster_resname, cluster_chainID in\
        zip(resSeq_of_each_contact, resname_of_each_contact, chainID_of_each_contact):
        for resSeq, resname, chainID in\
            zip(cluster_resSeq, cluster_resname, cluster_chainID):
            if resname != 'TYR':
                continue
            if chainID in chains[pdb_idcode].antibody:
                hydro_list_name_Seq_ID_ab.append(resname + chainID + str(resSeq))
            elif chainID in chains[pdb_idcode].antigen:
                hydro_list_name_Seq_ID_ag.append(resname + chainID + str(resSeq))
            else:
                logging.warning(f"This shouldn't happen")
    
    ##### 
    # Ring
    #####
    # Anion Ab
    Anion_list_name_Seq_ID_ab = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiAnion_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiAnion_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antibody]
    
    # Anion Ag
    Anion_list_name_Seq_ID_ag = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiAnion_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiAnion_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antigen]
    
    # Cation Ab
    Cation_list_name_Seq_ID_ab = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiCation_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiCation_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antibody]
    
    # Cation Ag
    Cation_list_name_Seq_ID_ag = [ pair_resname.ring + pair_chainID.ring + str(pair_resSeq.ring)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiCation_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiCation_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.ring == 'TYR' and pair_chainID.ring in chains[pdb_idcode].antigen]
    
    # Pi Ab
    Pi_list_name_Seq_ID_ab = [ pair_resname.antibody + pair_chainID.antibody + str(pair_resSeq.antibody)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiPi_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiPi_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.antibody == 'TYR' and pair_chainID.antibody in chains[pdb_idcode].antibody]
    
    # Pi Ag
    Pi_list_name_Seq_ID_ag = [ pair_resname.antigen + pair_chainID.antigen + str(pair_resSeq.antigen)\
    for pair_resname, pair_chainID, pair_resSeq in zip(
        df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0],
            df_PiPi_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0],
            df_PiPi_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0])\
    if pair_resname.antigen == 'TYR' and pair_chainID.antigen in chains[pdb_idcode].antigen]
    
    ##### 
    # Hbond
    #####
    list_resSeq = []
    list_resname = []
    list_chainID = []
    list_wat_resSeq = []
    list_wat_resname = []
    list_wat_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
            if Htuple.acceptor.resname == 'HOH' or Htuple.donor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.acceptor.resSeq)
                list_wat_resname.append(Htuple.acceptor.resname)
                list_wat_chainID.append(Htuple.acceptor.chainID)
            else:
                list_resSeq.append(Htuple.acceptor.resSeq)
                list_resname.append(Htuple.acceptor.resname)
                list_chainID.append(Htuple.acceptor.chainID)

    # No WAT
    hbond_list_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_resname, list_resSeq, list_chainID):
        hbond_list_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    # WAT
    hbond_list_wat_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_wat_resname, list_wat_resSeq, list_wat_chainID):
        hbond_list_wat_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    ##### 
    # Combinations
    #####
    set_hydro_list_name_Seq_ID_ab = set(hydro_list_name_Seq_ID_ab)
    set_hydro_list_name_Seq_ID_ag = set(hydro_list_name_Seq_ID_ag)
    set_Anion_list_name_Seq_ID_ab = set(Anion_list_name_Seq_ID_ab)
    set_Anion_list_name_Seq_ID_ag = set(Anion_list_name_Seq_ID_ag)
    set_Cation_list_name_Seq_ID_ab = set(Cation_list_name_Seq_ID_ab)
    set_Cation_list_name_Seq_ID_ag = set(Cation_list_name_Seq_ID_ag)
    set_Pi_list_name_Seq_ID_ab = set(Pi_list_name_Seq_ID_ab)
    set_Pi_list_name_Seq_ID_ag = set(Pi_list_name_Seq_ID_ag)
    set_hbond_list_resname_resSeq_chainID = set(hbond_list_resname_resSeq_chainID)
    set_hbond_list_wat_resname_resSeq_chainID = set(hbond_list_wat_resname_resSeq_chainID)
    
    tyr_combis_hydro_ab['PiAnion'] += len(\
        set_hydro_list_name_Seq_ID_ab.intersection(set_Anion_list_name_Seq_ID_ab))
    tyr_combis_hydro_ab['PiCation'] += len(\
        set_hydro_list_name_Seq_ID_ab.intersection(set_Cation_list_name_Seq_ID_ab))
    tyr_combis_hydro_ab['PiPi'] += len(\
        set_hydro_list_name_Seq_ID_ab.intersection(set_Pi_list_name_Seq_ID_ab))
    tyr_combis_hydro_ab['Hbond'] += len(\
        set_hydro_list_name_Seq_ID_ab.intersection(set_hbond_list_resname_resSeq_chainID))
    tyr_combis_hydro_ab['Water mediated'] += len(\
        set_hydro_list_name_Seq_ID_ab.intersection(set_hbond_list_wat_resname_resSeq_chainID))
    tyr_combis_hydro_ab['Total'] += tyr_hydro_cnt_ab

    tyr_combis_hydro_ag['PiAnion'] += len(\
        set_hydro_list_name_Seq_ID_ag.intersection(set_Anion_list_name_Seq_ID_ag))
    tyr_combis_hydro_ag['PiCation'] += len(\
        set_hydro_list_name_Seq_ID_ag.intersection(set_Cation_list_name_Seq_ID_ag))
    tyr_combis_hydro_ag['PiPi'] += len(\
        set_hydro_list_name_Seq_ID_ag.intersection(set_Pi_list_name_Seq_ID_ag))
    tyr_combis_hydro_ag['Hbond'] += len(\
        set_hydro_list_name_Seq_ID_ag.intersection(set_hbond_list_resname_resSeq_chainID))
    tyr_combis_hydro_ag['Water mediated'] += len(\
        set_hydro_list_name_Seq_ID_ag.intersection(set_hbond_list_wat_resname_resSeq_chainID))
    tyr_combis_hydro_ag['Total'] += tyr_hydro_cnt_ag

for key, val in tyr_combis_hydro_ab.items():
    tyr_combis_hydro_ab[key] = val / tyr_combis_hydro_ab['Total']
for key, val in tyr_combis_hydro_ag.items():
    tyr_combis_hydro_ag[key] = val / tyr_combis_hydro_ag['Total']

del tyr_combis_hydro_ab['Total']
tyr_combis_hydro_ab['Only hydrophobic'] = 1 - sum(tyr_combis_hydro_ab.values())
del tyr_combis_hydro_ag['Total']
tyr_combis_hydro_ag['Only hydrophobic'] = 1 - sum(tyr_combis_hydro_ag.values())

### tyr_combinations_ratio_hydro_ab

In [68]:
df_tyr_combi = pd.DataFrame({"y": list(tyr_combis_hydro_ab.values())[:-1],
    "x":list(tyr_combis_hydro_ab.keys())[:-1]})

df_tyr_combi = px.bar(df_tyr_combi, x="x", y="y", title="antibody")
df_tyr_combi.update_xaxes(title = "")
df_tyr_combi.update_yaxes(
    title = "# of interacting TYRs / # of TYRs in hydrophobic interaction", range=[0, .2])

### tyr_combinations_ratio_hydro_ab_pie

In [69]:
fig = go.Figure(data = go.Pie(labels = list(tyr_combis_hydro_ab.keys()),
    values = list(tyr_combis_hydro_ab.values()), hole = .4))
fig.update_traces(marker={'colors': ['Peru', 'DarkMagenta', 'DarkCyan', 'Crimson',
    'Aqua', 'Grey']})
fig.update_layout(annotations=[dict(
    text='Ab', x=0.5, y=0.5, font_size=20, showarrow=False)])

### tyr_combinations_ratio_hydro_ag

In [70]:
df_tyr_combi = pd.DataFrame({"y": list(tyr_combis_hydro_ag.values())[:-1],
    "x":list(tyr_combis_hydro_ab.keys())[:-1]})

df_tyr_combi = px.bar(df_tyr_combi, x="x", y="y", title="antibody")
df_tyr_combi.update_xaxes(title = "")
df_tyr_combi.update_yaxes(
    title = "# of interacting TYRs / # of TYRs in hydrophobic interaction", range=[0, .2])

### tyr_combinations_ratio_hydro_ag_pie

In [71]:
fig = go.Figure(data = go.Pie(labels = list(tyr_combis_hydro_ag.keys()),
    values = list(tyr_combis_hydro_ag.values()), hole = .4))
fig.update_traces(marker={'colors': ['Peru', 'DarkMagenta', 'DarkCyan', 'Crimson',
    'Aqua', 'Grey']})
fig.update_layout(annotations=[dict(
    text='Ag', x=0.5, y=0.5, font_size=20, showarrow=False)])

-----

### Enrichment

In [137]:
with open(Path.joinpath(casa_dir, 'data', 'count_tot_pdb.pkl'), 'rb') as file:
    count_tot_pdb = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_resi_pdb.pkl'), 'rb') as file:
    count_resi_pdb = pickle.load(file)

In [138]:
enrichment = dict(zip(AA_LIST, itertools.repeat(0, len(AA_LIST))))
for pdb_idcode in pdb_list:
    
    for resi, cnt in enrichment.items():
        tot_resi = count_tot_pdb[pdb_idcode][resi]
        if tot_resi != 0:
            enrichment[resi] += (count_resi_pdb[pdb_idcode].antibody[resi] +\
                count_resi_pdb[pdb_idcode].antigen[resi]) / tot_resi
for resi, cnt in enrichment.items():
    enrichment[resi] = cnt / len(pdb_list)

In [139]:
# Get the number of residues of each type at the interface
inter_count_resi = dict(zip(AA_LIST, itertools.repeat(0, len(AA_LIST))))
for pdb_idcode in pdb_list:
    
    for resi, cnt in inter_count_resi.items():
        inter_count_resi[resi] += (count_resi_pdb[pdb_idcode].antibody[resi] +\
            count_resi_pdb[pdb_idcode].antigen[resi])

# Get the ratio between the number of interface residues to total number of residues
inter_count = sum([sum(count_resi_pdb[pdb_idcode].antibody.values()) +\
    sum(count_resi_pdb[pdb_idcode].antigen.values()) for pdb_idcode in pdb_list])
interface_size = inter_count / sum(count_tot.values())

# Get the ratio between the number of interface residues to total number of residues,
# for each type of residue
inter_ratio_resi = dict(zip(AA_LIST, itertools.repeat(0, len(AA_LIST))))
for resi in inter_ratio_resi.keys():
    inter_ratio_resi[resi] = inter_count_resi[resi] / count_tot[resi]

# Finally, get the enrichment of each type of residue at the interface
enrichment_resi = dict(zip(AA_LIST, itertools.repeat(0, len(AA_LIST))))
for resi in enrichment_resi.keys():
    enrichment_resi[resi] = (inter_ratio_resi[resi] / interface_size)


### residue_enrichment_at_the_interface

In [140]:
df_enrichment = pd.DataFrame({'AA': enrichment_resi.keys(),
    'Count': enrichment_resi.values()})
    
fig_enrichment = px.histogram(df_enrichment, x='AA', y='Count', 
    # color='color', barmode='group',
    height=400)
fig_enrichment.update_xaxes(title = "AA type")
fig_enrichment.update_yaxes(title = "# of AA at the interface / total # of AA")

In [141]:
with open(Path.joinpath(casa_dir, 'data', 'simple_SSE.pkl'), 'rb') as file:
    SSE = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'simple_SSE_count.pkl'), 'rb') as file:
    SSE_cnt = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_tot_chain_pdb.pkl'), 'rb') as file:
    count_tot_chain_pdb = pickle.load(file)

##### Get the count of the SSEs of each interacting residue and the count of SSEs of all the residues

In [142]:
hydro_count_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
tot_count_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
tot_ratio_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
hydro_ratio_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}

for pdb_idcode in pdb_list:
    
    # First, get the list of unique residues
    list_resname = list(itertools.chain.from_iterable(
        df_hydrophobic_resname.query(f"idcode == '{pdb_idcode}'").resname[0]))
    list_resSeq = list(itertools.chain.from_iterable(
        df_hydrophobic_resSeq.query(f"idcode == '{pdb_idcode}'").resSeq[0]))
    list_chainID = list(itertools.chain.from_iterable(
        df_hydrophobic_chain_ID.query(f"idcode == '{pdb_idcode}'").chain_ID[0]))

    sse_list_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_resname, list_resSeq, list_chainID):
        if chainID in chains[pdb_idcode].antigen:
            sse_list_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(sse_list_resname_resSeq_chainID)
    unique_resSeq = [ int(unique_[4:]) for unique_ in unique_residues ]

    # Now check the SSE of the residues and add them to the count
    for resSeq in unique_resSeq:
        sse = SSE[pdb_idcode][resSeq]
        hydro_count_resi_sse[sse] += 1
    
    # Now count the total
    for key in tot_count_resi_sse.keys():
        tot_count_resi_sse[key] += SSE_cnt[pdb_idcode][key]

# Get the ratio between the number of residues for each SSE
# and the total number of residues
tot_count = sum(tot_count_resi_sse.values())
for key in tot_ratio_resi_sse.keys():
        tot_ratio_resi_sse[key] = tot_count_resi_sse[key] / tot_count

# Get the ratio between the number of residues for each SSE
# and the total number of residues at the interface
tot_count_interface = sum(hydro_count_resi_sse.values())
for key in hydro_count_resi_sse.keys():
        hydro_ratio_resi_sse[key] = hydro_count_resi_sse[key] / tot_count_interface

In [ ]:
hbond_count_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
tot_count_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
tot_ratio_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
hbond_ratio_resi_sse = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
enrichment_sse = {'H': 0, 'E': 0, 'C': 0}

for pdb_idcode in pdb_list:
    
    list_resSeq = []
    list_resname = []
    list_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
            if Htuple.acceptor.resname != 'HOH':
                list_resSeq.append(Htuple.acceptor.resSeq)
                list_resname.append(Htuple.acceptor.resname)
                list_chainID.append(Htuple.acceptor.chainID)
            if Htuple.donor.resname != 'HOH':
                list_resSeq.append(Htuple.donor.resSeq)
                list_resname.append(Htuple.donor.resname)
                list_chainID.append(Htuple.donor.chainID)

    sse_list_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_resname, list_resSeq, list_chainID):
        if chainID in chains[pdb_idcode].antigen:
            sse_list_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(sse_list_resname_resSeq_chainID)
    unique_resSeq = [ int(unique_[4:]) for unique_ in unique_residues ]

    # Now check the SSE of the residues and add them to the count
    for resSeq in unique_resSeq:
        sse = SSE[pdb_idcode][resSeq]
        hbond_count_resi_sse[sse] += 1
    
    # Now count the total
    for key in tot_count_resi_sse.keys():
        tot_count_resi_sse[key] += SSE_cnt[pdb_idcode][key]

# Get the ratio between the number of residues for each SSE
# and the total number of residues
tot_count = sum(tot_count_resi_sse.values())
for key in tot_ratio_resi_sse.keys():
        tot_ratio_resi_sse[key] = tot_count_resi_sse[key] / tot_count

# Get the ratio between the number of residues for each SSE
# and the total number of residues at the interface
tot_count_interface = sum(hbond_count_resi_sse.values())
for key in hbond_count_resi_sse.keys():
        hbond_ratio_resi_sse[key] = hbond_count_resi_sse[key] / tot_count_interface

In [163]:
enrichment_sse_hydro = {'H': 0, 'E': 0, 'C': 0}
enrichment_sse_hbond = {'H': 0, 'E': 0, 'C': 0}

for sse in enrichment_sse_hydro.keys():
    enrichment_sse_hydro[sse] = (hydro_ratio_resi_sse[sse] - tot_ratio_resi_sse[sse]) * 100

for sse in enrichment_sse_hbond.keys():
    enrichment_sse_hbond[sse] = (hbond_ratio_resi_sse[sse] - tot_ratio_resi_sse[sse]) * 100

### residue_enrichment_interactions

In [167]:
df_enrichment = pd.DataFrame({'AA': list(enrichment_sse_hydro.keys()) +\
    list(enrichment_sse_hbond.keys()),
    'percentage': list(enrichment_sse_hydro.values()) +\
    list(enrichment_sse_hbond.values()),
    'selection': np.repeat(['hydrophobic','polar'], 3)})
    
fig_enrichment = px.bar(df_enrichment, x='AA', y='percentage', 
    color='selection', barmode='group')
fig_enrichment.update_xaxes(title = "AA type")
fig_enrichment.update_yaxes(title = "Percentage points difference with the average",
    range=[-10, 15])

----

### even more ring interactions

In [87]:
with open(Path.joinpath(casa_dir, 'data', 'count_aa.pkl'), 'rb') as file:
    count_aa = pickle.load(file)

In [88]:
phe_interactions = {'Hydrophobic': 0, 'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}


for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname)[0]

    #####
    # Hydrophobic
    #####
    # Now I have to make sure I'm counting pheosines, instead of their atoms.
    list_resSeq = [ resSeq for cluster_resSeq in resSeq_of_each_contact\
        for resSeq in cluster_resSeq ]
    list_resname = [ resname for cluster_resname in resname_of_each_contact\
        for resname in cluster_resname ]
    list_chainID = [ chainID for cluster_chainID in chainID_of_each_contact\
        for chainID in cluster_chainID ]

    list_resSeq_chainID = []
    for resSeq, chainID in zip(list_resSeq, list_chainID):
        list_resSeq_chainID.append(str(resSeq) + chainID)

    unique_residues = set(list_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[:-1]))\
        for unique_ in unique_residues ]
    phe_hydro_cnt = sum([ list_resname[i] == 'PHE' for i in unique_indices ])

    #####
    # Ring
    #####
    phe_Anion_cnt = sum([ par.ring == 'PHE' \
        for par in df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    phe_Cation_cnt = sum([ par.ring == 'PHE' \
        for par in df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    phe_PiPi_cnt = sum([ par.antibody == 'PHE' \
        for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0] ]) +\
    sum([ par.antigen == 'PHE' \
        for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    #####
    # Hbond
    #####
    list_resSeq = []
    list_resname = []
    list_chainID = []
    list_wat_resSeq = []
    list_wat_resname = []
    list_wat_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
            if Htuple.acceptor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.donor.resSeq)
                list_wat_resname.append(Htuple.donor.resname)
                list_wat_chainID.append(Htuple.donor.chainID)
            elif Htuple.donor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.acceptor.resSeq)
                list_wat_resname.append(Htuple.acceptor.resname)
                list_wat_chainID.append(Htuple.acceptor.chainID)
            else:
                list_resSeq.append(Htuple.acceptor.resSeq)
                list_resname.append(Htuple.acceptor.resname)
                list_chainID.append(Htuple.acceptor.chainID)
                list_resSeq.append(Htuple.donor.resSeq)
                list_resname.append(Htuple.donor.resname)
                list_chainID.append(Htuple.donor.chainID)

    # No WAT
    list_resSeq_chainID = []
    for resSeq, chainID in zip(list_resSeq, list_chainID):
        list_resSeq_chainID.append(str(resSeq) + chainID)

    unique_residues = set(list_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[:-1]))\
        for unique_ in unique_residues ]

    phe_hbond_cnt = sum([ list_resname[i] == 'PHE' for i in unique_indices ])
    # WAT
    hbond_list_wat_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_wat_resname, list_wat_resSeq, list_wat_chainID):
        hbond_list_wat_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(hbond_list_wat_resname_resSeq_chainID)
    unique_indices = [ list_wat_resSeq.index(int(unique_[4:]))\
        for unique_ in unique_residues ]

    phe_hbond_wat_cnt = sum([ list_wat_resname[i] == 'PHE' for i in unique_indices ])

    #####
    # Total
    #####
    tot_cnt = count_aa[pdb_idcode]['PHE'].heavy +\
        count_aa[pdb_idcode]['PHE'].light +\
        count_aa[pdb_idcode]['PHE'].antigen
    phe_interactions['PiAnion'] += phe_Anion_cnt
    phe_interactions['PiCation'] += phe_Cation_cnt
    phe_interactions['PiPi'] += phe_PiPi_cnt
    phe_interactions['Hydrophobic'] += phe_hydro_cnt
    phe_interactions['Water mediated'] += phe_hbond_wat_cnt
    phe_interactions['Hbond'] += phe_hbond_cnt
    phe_interactions['Total'] += tot_cnt

phe_interactions_ratio_tot = {}
for key, val in phe_interactions.items():
    phe_interactions_ratio_tot[key] = val / phe_interactions['Total']

phe_interactions_ratio_inter = {}
for key, val in phe_interactions.items():
    phe_interactions_ratio_inter[key] = val / count_resi['PHE']

### phe_interactions_ratio_inter

In [89]:
df_phe = pd.DataFrame({"y": list(phe_interactions_ratio_inter.values())[:-1],
    "x":list(phe_interactions_ratio_inter.keys())[:-1]})
    
fig_phe = px.bar(df_phe, x="x", y="y")
fig_phe.update_xaxes(title = "")
fig_phe.update_yaxes(title = "# of interacting PHE / # of interface PHEs", range=[0, 1])


In [90]:
trp_interactions = {'Hydrophobic': 0, 'PiPi': 0, 'PiAnion': 0, 'PiCation': 0,
    'Hbond': 0, 'Water mediated': 0, 'Total': 0}


for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname)[0]

    #####
    # Hydrophobic
    #####
    # Now I have to make sure I'm counting trposines, instead of their atoms.
    list_resSeq = [ resSeq for cluster_resSeq in resSeq_of_each_contact\
        for resSeq in cluster_resSeq ]
    list_resname = [ resname for cluster_resname in resname_of_each_contact\
        for resname in cluster_resname ]
    list_chainID = [ chainID for cluster_chainID in chainID_of_each_contact\
        for chainID in cluster_chainID ]

    list_resSeq_chainID = []
    for resSeq, chainID in zip(list_resSeq, list_chainID):
        list_resSeq_chainID.append(str(resSeq) + chainID)

    unique_residues = set(list_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[:-1]))\
        for unique_ in unique_residues ]
    trp_hydro_cnt = sum([ list_resname[i] == 'TRP' for i in unique_indices ])
    #####
    # Ring
    #####
    trp_Anion_cnt = sum([ par.ring == 'TRP' \
        for par in df_PiAnion_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    trp_Cation_cnt = sum([ par.ring == 'TRP' \
        for par in df_PiCation_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    trp_PiPi_cnt = sum([ par.antibody == 'TRP' \
        for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0] ]) +\
    sum([ par.antigen == 'TRP' \
        for par in df_PiPi_name.query(f"idcode == '{pdb_idcode}'").resname[0] ])
    #####
    # Hbond
    #####
    list_resSeq = []
    list_resname = []
    list_chainID = []
    list_wat_resSeq = []
    list_wat_resname = []
    list_wat_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
            if Htuple.acceptor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.donor.resSeq)
                list_wat_resname.append(Htuple.donor.resname)
                list_wat_chainID.append(Htuple.donor.chainID)
            elif Htuple.donor.resname == 'HOH':
                list_wat_resSeq.append(Htuple.acceptor.resSeq)
                list_wat_resname.append(Htuple.acceptor.resname)
                list_wat_chainID.append(Htuple.acceptor.chainID)
            else:
                list_resSeq.append(Htuple.acceptor.resSeq)
                list_resname.append(Htuple.acceptor.resname)
                list_chainID.append(Htuple.acceptor.chainID)
                list_resSeq.append(Htuple.donor.resSeq)
                list_resname.append(Htuple.donor.resname)
                list_chainID.append(Htuple.donor.chainID)
    # No WAT
    list_resSeq_chainID = []
    for resSeq, chainID in zip(list_resSeq, list_chainID):
        list_resSeq_chainID.append(str(resSeq) + chainID)

    unique_residues = set(list_resSeq_chainID)
    unique_indices = [ list_resSeq.index(int(unique_[:-1]))\
        for unique_ in unique_residues ]

    trp_hbond_cnt = sum([ list_resname[i] == 'TRP' for i in unique_indices ])
    # WAT
    hbond_list_wat_resname_resSeq_chainID = []
    for resname, resSeq, chainID in zip(list_wat_resname, list_wat_resSeq, list_wat_chainID):
        hbond_list_wat_resname_resSeq_chainID.append(resname + chainID + str(resSeq))

    unique_residues = set(hbond_list_wat_resname_resSeq_chainID)
    unique_indices = [ list_wat_resSeq.index(int(unique_[4:]))\
        for unique_ in unique_residues ]

    trp_hbond_wat_cnt = sum([ list_wat_resname[i] == 'TRP' for i in unique_indices ])

    #####
    # Total
    #####
    tot_cnt = count_aa[pdb_idcode]['TRP'].heavy +\
        count_aa[pdb_idcode]['TRP'].light +\
        count_aa[pdb_idcode]['TRP'].antigen
    trp_interactions['PiAnion'] += trp_Anion_cnt
    trp_interactions['PiCation'] += trp_Cation_cnt
    trp_interactions['PiPi'] += trp_PiPi_cnt
    trp_interactions['Hydrophobic'] += trp_hydro_cnt
    trp_interactions['Water mediated'] += trp_hbond_wat_cnt
    trp_interactions['Hbond'] += trp_hbond_cnt
    trp_interactions['Total'] += tot_cnt

trp_interactions_ratio_tot = {}
for key, val in trp_interactions.items():
    trp_interactions_ratio_tot[key] = val / trp_interactions['Total']

trp_interactions_ratio_inter = {}
for key, val in trp_interactions.items():
    trp_interactions_ratio_inter[key] = val / count_resi['TRP']

### trp_interactions_ratio_inter

In [91]:
df_trp = pd.DataFrame({"y": list(trp_interactions_ratio_inter.values())[:-1],
    "x":list(trp_interactions_ratio_inter.keys())[:-1]})
    
fig_trp = px.bar(df_trp, x="x", y="y")
fig_trp.update_xaxes(title = "")
fig_trp.update_yaxes(title = "# of interacting TRP / # of interface TRPs", range=[0, 1])


----

In [92]:
ser_interactions = {'Hydrophobic': 0, 'Hbond': 0, 'Both': 0, 'None': 0, 'Total': 0}

for pdb_idcode in pdb_list:
    chainID_of_each_contact = (df_hydrophobic_chain_ID.query(
        f"idcode == '{pdb_idcode}'").chain_ID)[0]
    resSeq_of_each_contact = (df_hydrophobic_resSeq.query(
        f"idcode == '{pdb_idcode}'").resSeq)[0]
    resname_of_each_contact = (df_hydrophobic_resname.query(
        f"idcode == '{pdb_idcode}'").resname)[0]

    #####
    # Hydrophobic
    #####
    # Now I have to make sure I'm counting serosines, instead of their atoms.
    hydro_list_resSeq = [ resSeq for cluster_resSeq in resSeq_of_each_contact\
        for resSeq in cluster_resSeq ]
    hydro_list_resname = [ resname for cluster_resname in resname_of_each_contact\
        for resname in cluster_resname ]
    hydro_list_chainID = [ chainID for cluster_chainID in chainID_of_each_contact\
        for chainID in cluster_chainID ]

    hydro_list_resSeq_chainID = []
    for resSeq, chainID in zip(hydro_list_resSeq, hydro_list_chainID):
        hydro_list_resSeq_chainID.append(str(resSeq) + chainID)

    hydro_unique_residues = set(hydro_list_resSeq_chainID)

    #####
    # Hbond
    #####
    hbond_list_resSeq = []
    hbond_list_resname = []
    hbond_list_chainID = []
    for bond_list in hbonds[pdb_idcode].values():
        for Htuple in bond_list:
                hbond_list_resSeq.append(Htuple.acceptor.resSeq)
                hbond_list_resname.append(Htuple.acceptor.resname)
                hbond_list_chainID.append(Htuple.acceptor.chainID)
                hbond_list_resSeq.append(Htuple.donor.resSeq)
                hbond_list_resname.append(Htuple.donor.resname)
                hbond_list_chainID.append(Htuple.donor.chainID)
        
    hbond_list_resSeq_chainID = []
    for resSeq, chainID in zip(hbond_list_resSeq, hbond_list_chainID):
        hbond_list_resSeq_chainID.append(str(resSeq) + chainID)

    hbond_unique_residues = set(hbond_list_resSeq_chainID)

    #####
    # Intersection and difference
    #####
    both_unique_residues = hbond_unique_residues.intersection(hydro_unique_residues)
    both_unique_indices = [ hydro_list_resSeq.index(int(unique_[:-1]))\
        for unique_ in both_unique_residues ]
    ser_both_cnt = sum([ hydro_list_resname[i] == 'SER' for i in both_unique_indices ])

    solo_hydro_unique_residues = hydro_unique_residues - hbond_unique_residues
    hydro_unique_indices = [ hydro_list_resSeq.index(int(unique_[:-1]))\
        for unique_ in solo_hydro_unique_residues ]
    ser_hydro_cnt = sum([ hydro_list_resname[i] == 'SER' for i in hydro_unique_indices ])

    solo_hbond_unique_residues = hbond_unique_residues - hydro_unique_residues
    hbond_unique_indices = [ hbond_list_resSeq.index(int(unique_[:-1]))\
        for unique_ in solo_hbond_unique_residues ]

    ser_hbond_cnt = sum([ hbond_list_resname[i] == 'SER' for i in hbond_unique_indices ])
    
    #####
    # Total
    #####
    tot_all = count_aa[pdb_idcode]['SER'].heavy +\
        count_aa[pdb_idcode]['SER'].light +\
        count_aa[pdb_idcode]['SER'].antigen
    tot_interface = count_resi_pdb[pdb_idcode].antibody['SER'] +\
        count_resi_pdb[pdb_idcode].antigen['SER']
    ser_interactions['Hydrophobic'] += ser_hydro_cnt
    ser_interactions['Hbond'] += ser_hbond_cnt
    ser_interactions['Both'] += ser_both_cnt
    ser_interactions['Total'] += tot_interface
    ser_interactions['None'] += (tot_interface - ser_both_cnt - ser_hbond_cnt - ser_hydro_cnt)

ser_interactions_ratio_inter = {}
for key in ser_interactions.keys():
    ser_interactions_ratio_inter[key] = ser_interactions[key] / count_resi['SER']

### serine

In [93]:
fig_ser = go.Figure(data = go.Pie(labels = ['Hydrophobic', 'Hydrogen bond',
'Both', 'None'], 
values = [ser_interactions_ratio_inter['Hydrophobic'], 
ser_interactions_ratio_inter['Hbond'], ser_interactions_ratio_inter['Both'],
ser_interactions_ratio_inter['None']], hole = .4))

fig_ser.update_traces(marker={'colors': ['BurlyWood', 'Crimson', 'DarkMagenta', 'Grey']})
fig_ser.update_layout(annotations=[dict(
    text='SER', x=0.5, y=0.5, font_size=20, showarrow=False)])

In [132]:
df_sse_epi = pd.DataFrame({'sse': list(sse_epi_all_ratio.keys()) + list(sse_epi_interface_ratio.keys()), 
    'probability': list(sse_epi_all_ratio.values()) + list(sse_epi_interface_ratio.values()) ,
    'selection': np.repeat(['whole antigen','antigen interface'], 3)})
    
px.bar(df_sse_epi, x='sse', y='probability', color='selection', barmode='group')

In [166]:
df_enrichment = pd.DataFrame({'AA': list(enrichment_sse_hydro.keys()) +\
    list(enrichment_sse_hbond.keys()),
    'percentage': list(enrichment_sse_hydro.values()) +\
    list(enrichment_sse_hbond.values()),
    'selection': np.repeat(['hydrophobic','polar'], 3)})
    
fig_enrichment = px.bar(df_enrichment, x='AA', y='percentage', 
    color='selection', barmode='group')
fig_enrichment.update_xaxes(title = "AA type")
fig_enrichment.update_yaxes(title = "Percentage points difference with the average",
    range=[-10, 15])

In [168]:
cdr_colors

['#74D2F5', '#5495D6', '#3763F5', '#CF81EB', '#E564F5', '#A04AD6']